# Practical Quantization in PyTorch

## FUNDAMENTALS OF QUANTIZATION

Quantization has roots in information compression; in deep networks it refers to reducing the numerical precision of its weights and/or activations.

Overparameterized DNNs have more degrees of freedom and this makes them good candidates for information compression. When you quantize a model, two things generally happen - the model gets smaller and runs with better efficiency. 

- Hardware vendors explicitly allow for faster processing of 8-bit data (than 32-bit data) resulting in higher throughput. 
- A smaller model has lower memory footprint and power consumption, crucial for deployment at the edge.

### Mapping function

The mapping function is what you might guess - a function that maps values from floating-point to integer space. A commonly used mapping function is a linear transformation given by:

$$Q(r)=round(r/S + Z)$$

where `r` is the input and `S` are `Z` `quantization parameters`.

To reconvert to floating point space, the inverse function is given by:

$$\hat r=(Q(r) - Z)*S$$

$\hat r \neq r$, and their difference constitutes the quantization error.

### Quantization Parameters

The mapping function is parameterized by the scaling factor `S` and zero-point `Z`.

 `S` is simply the ratio of the input range to the output range:

 $$ S = \dfrac {\beta - \alpha} {\beta_q - \alpha_q}$$

where [$\alpha$, $\beta$] is the clipping range of the input, i.e. the boundaries of permissible inputs. [$\alpha_q$, $\beta_q$] is the range in quantized output space that it is mapped to. For 8-bit quantization, the output range:
$$ \beta_q - \alpha_q <= (2^8 - 1)$$

`Z` acts as a bias to ensure that a 0 in the input space maps perfectly to a 0 in the quantized space:
$$Z=-(\dfrac {\alpha}{S} - \alpha_q)$$ 

### Calibration

The process of choosing the input clipping range is known as calibration. The simplest technique (also the default in PyTorch) is to record the running mininmum and maximum values and assign them to $\alpha$ and $\beta$. `TensorRT` also uses entropy minimization (KL divergence), mean-square-error minimization, or percentiles of the input range.

In PyTorch, `Observer` modules collect statistics on the input values and calculate the qparams `S`, `Z`. Different calibration schemes result in different quantized outputs, and it’s best to empirically verify which scheme works best for your application and architecture (more on that later).

In [3]:
import torch
from torch.quantization.observer import MinMaxObserver, MovingAverageMinMaxObserver, HistogramObserver

C, L = 3, 4
normal = torch.distributions.normal.Normal(0, 1)
inputs = [normal.sample((C, L)), normal.sample((C, L))]
print(inputs)

[tensor([[-1.3661,  0.6686, -0.7628, -0.1713],
        [ 0.5461, -0.9687,  0.5894, -0.1433],
        [ 0.0744,  1.8963, -0.9171,  0.0334]]), tensor([[ 0.3307, -1.0928, -0.8456,  1.2337],
        [ 0.6180, -1.6796, -0.9699,  0.8957],
        [ 0.5770, -1.1889,  0.9793,  1.6308]])]


In [3]:
observers = [MinMaxObserver(), MovingAverageMinMaxObserver(), HistogramObserver()]
for obs in observers:
  for x in inputs:
    obs(x)
  print(obs.__class__.__name__, obs.calculate_qparams())

MinMaxObserver (tensor([0.0185]), tensor([152], dtype=torch.int32))
MovingAverageMinMaxObserver (tensor([0.0139]), tensor([137], dtype=torch.int32))
HistogramObserver (tensor([0.0128]), tensor([159], dtype=torch.int32))


### Affine and Symmetric Quantization Schemes

__Affine or asymmetric quantization__ schemes assign the input range to the min and max observed values. Affine schemes generally offer tighter clipping ranges and are useful for quantizing non-negative activations (you don’t need the input range to contain negative values if your input tensors are never negative). The range is calculated as $\alpha=min(r)$, $\beta=max(r)$. Affine quantization leads to more computationally expensive inference when used for weight tensors.

__Symmetric quantization__ schemes center the input range around 0, eliminating the need to calculate a zero-point offset. The range is calculated as:

$$ -\alpha = \beta = max(|max(r)|, |min(r)|)$$

 For skewed signals (like non-negative activations) this can result in bad quantization resolution because the clipping range includes values that never show up in the input.

 In PyTorch, you can specify affine or symmetric schemes while initializing the Observer. Note that not all observers support both schemes.

In [4]:
for qscheme in [torch.per_tensor_affine, torch.per_tensor_symmetric]:
  obs = MovingAverageMinMaxObserver(qscheme=qscheme)
  for x in inputs:
    obs(x)
  print(f"Qscheme: {qscheme} | {obs.calculate_qparams()}")

Qscheme: torch.per_tensor_affine | (tensor([0.0128]), tensor([107], dtype=torch.int32))
Qscheme: torch.per_tensor_symmetric | (tensor([0.0149]), tensor([128]))


### Per-Tensor and Per-Channel Quantization Schemes

Quantization parameters can be calculated for the layer’s entire weight tensor as a whole, or separately for each channel. In per-tensor, the same clipping range is applied to all the channels in a layer

<img src="pic/per-channel-tensor.svg">